In [3]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [4]:
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
df.shape

(103, 3)

In [6]:
def getLat(neighborhood):
    address = '{} Toronto, Canada'.format(neighborhood)
    geolocator = Nominatim(user_agent="ca_explorer")
    location = geolocator.geocode(address)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
        return(latitude)
    else:
        return(float('NaN'))
df['Latitude']= df['Neighborhood'].apply(lambda x: getLat(x))

def getLong(neighborhood):
    address = '{} Toronto, Canada'.format(neighborhood)
    geolocator = Nominatim(user_agent="ca_explorer")
    location = geolocator.geocode(address)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
        return(longitude)
    else:
        return(float('NaN'))
df['Longitude']= df['Neighborhood'].apply(lambda x: getLong(x))

In [7]:
df.dropna(inplace = True)

In [8]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.640080,-79.380150
5,M9A,Etobicoke,Islington Avenue,43.688307,-79.542802
6,M1B,Scarborough,"Malvern, Rouge",43.809196,-79.221701
7,M3B,North York,Don Mills North,43.775347,-79.345944
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.712452,-79.312653
10,M6B,North York,Glencairn,43.708712,-79.440685
13,M3C,North York,Don Mills South,43.775347,-79.345944
14,M4C,East York,Woodbine Heights,43.699920,-79.319279
